Welcome to your DataCamp project audition! This notebook must be filled out and vetted before a contract can be signed and you can start creating your project.

The first step is forking the repository in which this notebook lives. After that, there are two parts to be completed in this notebook:

- **Project information**:  The title of the project, a project description, etc.

- **Project introduction**: The three first text and code cells that will form the introduction of your project.

When complete, please email the link to your forked repo to projects@datacamp.com with the email subject line _DataCamp project audition_. If you have any questions, please reach out to projects@datacamp.com.

# Project information

**Project title**: Predicting Football Results

**Name:** Michał Oleszak

**Email address associated with your DataCamp account:** oleszak.michal@gmail.com

**Project description**: Football, also known as soccer in the U.S., is the most popular sport in the world, with some reports estimating the total of over 4 billion fans. That's more than half of the world's population! Among the major football leagues, the English Premier League is often considered the strongest: it boasts most spectators in the stadiums, largest revenue and a very high competition level - even less recognized clubs stand a chance of winning against top teams. Is it possible to predict the results of such a fiercely competitive league? This project leverages powerful machine learning and deep learning techniques to try to achievie just that!

Before starting this project you should be comfortable manipulating data frames using the `dplyr` pipelines and the `apply`-family functions, and have some experience with the `tidyverse` packages `ggplot2` and `tidyr`. For the modelling we will use `caret` and `keras` libraries, but no prior knowledge of these two is assumed.

This project uses two datasets: past Premier League games records available via the [football-data API](http://www.football-data.co.uk/mmz4281/1819/E0.csv) and the ELO ratings of clubs, downloaded using the [clubelo API](http://clubelo.com/API).

# Project introduction

***Note: nothing needs to be filled out in this cell. It is simply setting up the template cells below.***

The final output of a DataCamp project looks like a blog post: pairs of text and code cells that tell a story about data. The text is written from the perspective of the data analyst and *not* from the perspective of an instructor on DataCamp. So, for this blog post intro, all you need to do is pretend like you're writing a blog post -- forget the part about instructors and students.

Below you'll see the structure of a DataCamp project: a series of "tasks" where each task consists of a title, a **single** text cell, and a **single** code cell. There are 8-12 tasks in a project and each task can have up to 10 lines of code. What you need to do:
1. Read through the template structure.
2. As best you can, divide your project as it is currently visualized in your mind into tasks.
3. Fill out the template structure for the first three tasks of your project.

As you are completing each task, you may wish to consult the project notebook format in our [documentation](https://instructor-support.datacamp.com/projects/datacamp-projects-jupyter-notebook). Only the `@context` and `@solution` cells are relevant to this audition.

## 1. Load and inspect the data

A cliché has it that the English Premier League games are hardest to predict. There is some truth to that: compare it to the Spanish La Liga, where betting on Real Madrid or FC Barcelona to become champions would have got you correct 13 times in the last 14 seasons. England, on the other hand, has seen four different teams winning the title interchangeably in the same period, with the most unexpected being Leicester City finishing first in 2016, having been promoted to the highest division just two years earlier. In this notebook, we will take on the challenge to forecast Premier League results using a set of machine- and deep learining techniques. Let's dive straight into it!

We will be using two data sets in this notebook. First, `past_games_data` containes one row per match with match date, home team, away team, goals scored by both teams and the end result as columns. Second, `elo_data` contains the so-called elo-score for each team and date. The elo-score is a rating capturing the strength of the team, and the elo difference between two clubs reflects the win probability of both teams in a direct confrontation. This should make it a useful predictor of the final result!

In [ ]:
# Load packages
library(readr)
library(dplyr)

# Import the data from CSV files
past_games_data <- read_csv("datasets/past_games_data.csv")
elo_data <- read_csv("datasets/elo_data.csv")

# Look at the top few rows of both dataframes
head(past_games_data)
head(elo_data)

## 2. Write a function to calculate recent form

The elo-scores might have good predictive power, but they don't capture everything. We will need to construct more features from available data. What, apart from the club's quality, can impact their winning chance? One of such things is the recent form. We can approximate it, for instance, by the number of wins in the last couple of games.

However, in order to be able to train the models on historic data, we will need not only the current form of each club, but also what we can call historic recent form: what the recent form has been on each match date for the involved teams. To compute it, let us first write a function that takes games data such as `past_games_data` and club names as inputs, and returns the recent form for these clubs. Later we will apply this function to all game dates!

In [ ]:
get_recent_form <- function(games, clubs) {
  # To each club, apply a function that returns the number of their recent wins
  lapply(clubs, function(club) {
    games %>% 
      # Filter on games that featured the club currently being looped over
      filter(home_team == club | away_team == club) %>%
      # Look at last 10 games only
      tail(10) %>% 
      # For each club calculate the number of home and away games won
      mutate(club = club) %>% 
      group_by(club) %>% 
      summarise(recent_home_games_won = sum((result == "H")[home_team == club]),
                recent_away_games_won = sum((result == "A")[away_team == club]))
    }) %>%
    # lapply returns a list of clubs; coerce it to data frame by binding rows
    bind_rows()
}

## 3. Get recent form for all teams at all times

With the function we have just written, we can now calculate the historic recent form on each match date for all involved clubs! We will do this in form of an elegant loop that comprises the following steps:

* Loop over all dates when games were played;
* For each date, filter the data to only keep games played before that date (games played later in the future have no relation with the recent form on that particular day);
* Based on thus filtered data, use our custom function we have just written to calculate the then-recent form for the clubs that played on that day.

In [ ]:
# Get unique clubs featured in the data
clubs <- unique(c(past_games_data[["home_team"]], past_games_data[["away_team"]]))

# Get unique dates on which games were played
unique_dates <- unique(past_games_data[["date"]])

# For each date calculate then-recent form of the clubs that played on this day                      
recent_form <- lapply(unique_dates, function(game_date) {
  # Get clubs that played on this date
  clubs_involved <- unique(c(past_games_data$home_team[past_games_data$date == game_date],
                             past_games_data$away_team[past_games_data$date == game_date]))
  past_games_data %>%
    # Filter out dates later than the date being looped over
    filter(date < game_date) %>% 
    # Calculate recent form using previously defined function
    get_recent_form(clubs = clubs_involved) %>%
    # Add column with date for which recent form data is valid
    mutate(date = game_date)
}) %>% 
  # lapply returns a list; coerce it to data frame by binding rows
  bind_rows()

*Stop here! Only the three first tasks. :)*